In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

import plotly as plt
from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import regex as re
import requests
from sklearn.svm import SVC

Load all files.

In [2]:
data_dir = '../data/01_raw/bbc/'
business, entertainment, politics, sport, tech = [], [], [], [], []
for file in os.listdir(data_dir + '/business/'):
    with open(data_dir + '/business/' + file, 'r') as f:
        business.append(f.read())
        
for file in os.listdir(data_dir + '/entertainment/'):
    with open(data_dir + '/entertainment/' + file, 'r') as e:
        entertainment.append(e.read())
        
for file in os.listdir(data_dir + '/politics/'):
    with open(data_dir + '/politics/' + file, 'r') as e:
        politics.append(e.read())
                
for file in os.listdir(data_dir + '/sport/'):
    with open(data_dir + '/sport/' + file, 'r') as e:
        sport.append(e.read())

for file in os.listdir(data_dir + '/tech/'):
    with open(data_dir + '/tech/' + file, 'r') as e:
        tech.append(e.read())

In [3]:
print ('Length of businssess {}, \n length of entertainment {} \n length politics: {}, length sport {}, \n length tech {}'.format(len(business), len(entertainment), len(politics), len(sport), len(tech)))

Length of businssess 510, 
 length of entertainment 386 
 length politics: 417, length sport 511, 
 length tech 401


In [4]:
def tokenize(x):
 tokenizer = RegexpTokenizer(r'\w+')
 return tokenizer.tokenize(x)


In [6]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Add the TextCategorizer to the empty model
textcat = nlp.add_pipe("textcat")
# textcat = nlp.remove_pipe('lemmatizer')
# Add labels to text classifier
textcat.add_label("business")
textcat.add_label("tech")
textcat.add_label("politics")
textcat.add_label("entertainment")
textcat.add_label("sport")

1

### Training a Text Categorizer Model


In [7]:
business_df  = pd.DataFrame({'text': business})
business_df.insert(0, 'label', 'business', True)
entertainment_df  = pd.DataFrame({'text': entertainment})
entertainment_df.insert(0, 'label', 'entertainment')
politics_df  = pd.DataFrame({'text': politics})
politics_df.insert(0, 'label', 'politics')
sport_df = pd.DataFrame({'text': sport})
sport_df.insert(0, 'label', 'sport')
tech_df = pd.DataFrame({'text': tech})
tech_df.insert(0, 'label', 'tech')

In [8]:
all_texts = pd.concat([business_df, politics_df, sport_df, tech_df, entertainment_df])
all_texts.to_csv('../data/02_intermediate/all_text_labels.csv')

Train Test Split

In [9]:
all_texts = pd.read_csv('../data/02_intermediate/all_text_labels.csv')
all_texts = all_texts.drop(['Unnamed: 0'], axis=1)
all_texts.head()

,label,text
0,business,Ad sales boost Time Warner profit\n\nQuarterly...
1,business,Dollar gains on Greenspan speech\n\nThe dollar...
2,business,Yukos unit buyer faces loan claim\n\nThe owner...
3,business,High fuel prices hit BA's profits\n\nBritish A...
4,business,Pernod takeover talk lifts Domecq\n\nShares in...


In [73]:
from sklearn.model_selection import train_test_split
X = all_texts.drop('label',axis=1)
y = all_texts['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [74]:
y_test

717     politics
798     politics
1330       sport
18      business
1391       sport
          ...   
296     business
1701        tech
781     politics
1359       sport
1836        tech
Name: label, Length: 668, dtype: object

In [16]:
train_texts = X_train['text'].values
train_labels = [{'cats': {'business': label == 'business',
                          'entertainment':label == 'entertainment',
                          'politics' : label == 'politics',
                          'sport': label == 'sport',
                          'tech': label == 'tech'}} for label in y_train]

In [18]:
# combine the texts and labels into a single list.
train_data = list(zip(train_texts, train_labels))
train_data[-3]

('Intel unveils laser breakthrough\n\nIntel has said it has found a way to put a silicon-based laser on a chip, raising hopes of much faster networks.\n\nScientists at Intel have overcome a fundamental problem that before now has prevented silicon being used to generate and amplify laser light. The breakthrough should make it easier to interconnect data networks with the chips that process the information. The Intel researchers said products exploiting the breakthrough should appear by the end of the decade.\n\n"We\'ve overcome a fundamental limit," said Dr Mario Paniccia, director of Intel\'s photonics technology lab. Writing in the journal Nature, Dr Paniccia - and colleagues Haisheng Rong, Richard Jones, Ansheng Liu, Oded Cohen, Dani Hak and Alexander Fang - show how they have made a continuous laser from the same material used to make computer processors. Currently, says Dr Paniccia, telecommunications equipment that amplifies the laser light that travels down fibre optic cables is

First, create an optimizer using `nlp.begin_training()`. spaCy uses this optimizer to update the model. In general it's more efficient to train models in small batches. spaCy provides the minibatch function that returns a generator <u>yielding minibatches</u> for training. Finally, the minibatches are split into texts and labels, then used with nlp.update to update the model's parameters.

In [71]:
from spacy.util import minibatch
from spacy.training.example import Example

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) 
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

This is using loop to reshuffle training data at the beginning of each loop.

In [22]:
from spacy.util import minibatch
from spacy.training.example import Example
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 568.5723469678775}
{'textcat': 719.8168045304549}
{'textcat': 788.481938169019}
{'textcat': 836.7500243831912}
{'textcat': 866.7167715687638}
{'textcat': 896.268980501044}
{'textcat': 913.9506380312018}
{'textcat': 931.9563007129077}
{'textcat': 949.2734642733325}
{'textcat': 955.4252219810468}


Predict Test Set

In [23]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [28]:
docs = [nlp.tokenizer(text) for text in X_test['text']]

# use Textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)
print(scores.shape)

(668, 5)


The scores are used to predict a single class or label by choosing the label with the highest probability. You get the index of the highest probability with scores.argmax, then use the index to get the label string from textcat.labels.

In [42]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
# predicted_labels
predictions = np.array([textcat.labels[label] for label in predicted_labels]).reshape(-1)

In [72]:
for i in range(5):
    print (i, textcat.labels[i])

0 business
1 tech
2 politics
3 entertainment
4 sport


In [75]:
y_test.replace({'business': 0, 'tech': 1, 'politics': 2, 'entertainment': 3, 'sport': 4}, inplace=True)

In [76]:
print('logloss %0.3f' %multiclass_logloss(y_test, scores))

logloss 0.410


In [77]:
import joblib
# saving our model # model - model , filename-model_jlib
joblib.dump(optimizer , '../data/06_models/nlp_model.joblib')

['../data/06_models/nlp_model.joblib']

## Data Preprocessing
1. Encoding
2. Source : https://www.kaggle.com/code/abhishek/approaching-almost-any-nlp-problem-on-kaggle

In [94]:
import xgboost as xgb
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


We use the LabelEncoder from scikit-learn to convert text labels to integers, 0, 1 2

In [83]:
from sklearn.model_selection import train_test_split


all_texts = pd.read_csv('../data/02_intermediate/all_text_labels.csv')
all_texts = all_texts.drop(['Unnamed: 0'], axis=1)

X = all_texts.drop('label',axis=1)
y = all_texts['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [86]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(y_train.values)

In [87]:
# Split to get train and validation
xtrain, xvalid, ytrain, yvalid = train_test_split(X_train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [88]:
print (xtrain.shape)
print (xvalid.shape)

(1401,)
(156,)


Basic Model (TF_IDF)

In [89]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [90]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.446 


Count-Vectorizer Model

In [91]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [92]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.073 


- Compared to TF-IDF (0.446) and Count Vectoriser (0.073), Logistic regression has improved a lot the model

## Test on XgBoost
 use CV.

In [95]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

D:\1. Data Science\2. 2022\[APR22] HACKATHON\hack3.9\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:35:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.080 


Seems really good XGBoost Classifier (0.080), slightly worse that the Logistic regression of Count vectoriser.
- may consider Logistic regression over this, since XGB model is slightly complex, compared to logistic regression.